In [2]:
import torch
from src.model.spex_plus import SpExPlus
from src.datasets.custom_dir_audio_dataset import CustomDirAudioDataset
from src.metric.utils import si_sdr
import pesq
model_path = "saved/models/default_config/1108_065537/checkpoint-epoch80.pth"
dataset = CustomDirAudioDataset("/home/vladimir/PycharmProjects/TTS/mixtures_data/test_clean",None, None)
model = SpExPlus(num_speakers=96)
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu'))['state_dict'])

<All keys matched successfully>

In [3]:
class NewDataset(CustomDirAudioDataset):
    def __getitem__(self, idx):
        super_res = super().__getitem__(idx)
        new_d = dict()
        for k, v in super_res.items():
            if hasattr(v, 'shape'):
                new_d[f'{k}_len'] = torch.tensor([v.shape[1]])
                
        super_res.update(new_d)
        return super_res
dataset = NewDataset("/home/vladimir/PycharmProjects/TTS/mixtures_data/test_clean/61",None, None)

In [13]:
from torc

NameError: name 'pad_sequence' is not defined

In [5]:
model(**dataset[0])['s1']

tensor([[-0.5124, -0.2580, -0.1654,  ..., -0.4899, -0.5053, -0.4778]],
       grad_fn=<SqueezeBackward1>)

In [ ]:
model(**dataset[0])['s1']

In [39]:
mix_audio = obj['mix_audio']
frame_size = 3 * 16000 
padded_audio = torch.nn.functional.pad(mix_audio, (0, frame_size - mix_audio.size(1) % frame_size))
mix_audio = padded_audio.reshape(-1, frame_size)

In [68]:
target_audio = obj['target_audio']
padded_audio = torch.nn.functional.pad(target_audio, (0, frame_size - target_audio.size(1) % frame_size))
target_audio = padded_audio.reshape(-1, frame_size)

In [53]:
reference_audio = obj['reference_audio'].repeat(len(mix_audio), 1)
reference_audio_len = torch.tensor([reference_audio.shape[1]]).repeat(len(mix_audio), 1).flatten()

In [54]:
pred = model(mix_audio, reference_audio, reference_audio_len)['s1'].detach().numpy()
target = target_audio.detach().numpy()

In [55]:
pesq.pesq(16_000, pred.flatten(), target.flatten())

1.1306016445159912

In [78]:
audio_len = obj['target_audio_len'].item()

In [75]:
import numpy as np

def si_sdr(est, target):
    alpha = np.sum(target * est) / np.linalg.norm(target) ** 2
    return 20 * np.log10(
        np.linalg.norm(alpha * target)
        / (np.linalg.norm(alpha * target - est) + 1e-6)
        + 1e-6
    )

In [81]:
si_sdr(pred.flatten()[:audio_len // 2], target.flatten()[:audio_len // 2])

1.661399735589872

In [82]:
si_sdr(pred.flatten()[audio_len // 2:], target.flatten()[audio_len // 2:])

-5.556198851344872